In [0]:
%%capture
%tensorflow_version 1

## **Install rdkit and deepchem**

In [0]:
!apt-get -qq install -y python-rdkit librdkit1 rdkit-data

In [0]:
!pip install -q joblib pandas sklearn tensorflow pillow deepchem

In [0]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from time import time
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
import random

import pandas as pd
import io
import requests
import warnings

import deepchem as dc
import deepchem.models.tensorgraph.layers as layers
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten

warnings.filterwarnings("ignore")    

In [0]:
# Global variable 
NUMBER_CHEMICAL_SHIFT_TYPE = 19

def get_cs_all(cs_all, id = "2KOC"):
  '''    
    This function gets chemical shifts for a particular RNA. 
    Assumes each RNA has a unique id  
  '''
  return(cs_all[(cs_all.id == id)])

def get_cs_residues(cs_i, resid, dummy = 0):
  '''    
    This function return an array containing the chemical shifts for a particular residues in an RNA.    
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)].drop(['id', 'resid', 'resname', 'stacking'], axis=1)
  info_tmp = cs_i[(cs_i.resid == resid)]
  if (cs_tmp.shape[0] != 1):
     return(dummy*np.ones(shape=(1, NUMBER_CHEMICAL_SHIFT_TYPE)))
  else:
     return(cs_tmp.values)
    
def get_resnames(cs_i, resid, dummy = "UNK"):
  '''    
    This function returns the residue name for specified residue (resid)
  '''
  cs_tmp = cs_i[(cs_i.resid == resid)]  
  if (cs_tmp.shape[0] != 1):
     return(dummy)
  else:
     return(cs_tmp['resname'].values[0])

def get_cs_features(cs_i, resid, neighbors=1):
  '''    
  This function chemical shifts and resnames for residue (resid) and its neighbors        

  '''
  cs = []
  resnames = []
  for i in range(resid-neighbors, resid+neighbors+1):
    cs.append(get_cs_residues(cs_i, i))
    resnames.append(get_resnames(cs_i, i))
  return(resnames, np.array(cs))

def get_columns_names(neighbors = 3, chemical_shift_types = 19):
  '''
    
    Helper function that writes out the required column names
    
  '''

  columns = ['id', 'resname', 'resid', 'Stacking']
  for i in range(0, neighbors*chemical_shift_types):
    columns.append(i)
  return(columns)

def write_out_resname(neighbors=1):
  '''
  
    Helper function that writes out the column names associated resnames for a given residue and its neighbors
    
  '''  
  colnames = []
  for i in range(1-neighbors-1, neighbors+1):
    if i < 0: 
      colnames.append('R%s'%i)
    elif i > 0: 
      colnames.append('R+%s'%i)
    else: 
      colnames.append('R')
  return(colnames)    


def get_cs_features_rna(cs, neighbors=1, retain = ['id', 'stacking', 'resid']):
  '''    
    This function generates the complete required data frame an RNA    
  '''
  all_features = []
  all_resnames = []
  for resid in cs['resid'].unique():
    resnames, features = get_cs_features(cs, resid, neighbors)
    all_features.append(features.flatten())
    all_resnames.append(resnames)

  all_resnames = pd.DataFrame(all_resnames, dtype='object', columns = write_out_resname(neighbors))
  all_features = pd.DataFrame(all_features, dtype='object')
  info = pd.DataFrame(cs[retain].values, dtype='object', columns = retain)
  return(pd.concat([info, all_resnames, all_features], axis=1))

def get_cs_features_rna_all(cs, neighbors = 2):  
  '''    
    This [should] function generate a pandas dataframe containing training data for all RNAs
    Each row in the data frame should contain the class and chemical shifts for given residue and neighbors in a given RNA.
    Use the function above to write function
    
  '''  
  # Start: your code
  ids = cs['id'].unique()
  for i,id in enumerate(ids):
    if i == 0:
      cs_new = get_cs_features_rna(get_cs_all(cs, id), neighbors)
    else:
      cs_new = cs_new.append(get_cs_features_rna(get_cs_all(cs, id), neighbors), sort = False)        
  # End: your code
  return(cs_new)

def create_training_testing(cs, leave_out = "2KOC", target_name = 'stacking', neighbors = 2, drop_names = ['id', 'stacking', 'resid']):
  '''    
    This function creates a training and testing set using leave one out    
  '''
  
  # drop extraneous data  
  drop_names = drop_names + list(write_out_resname(neighbors))  
  
  # does not contain leave_out
  train = cs[(cs.id != leave_out)]
  trainX = train.drop(drop_names, axis=1)
  trainy = train[target_name]
 
  # only contains leave_out
  test = cs[(cs.id == leave_out)]
  testX = test.drop(drop_names, axis=1)
  testy = test[target_name]
  
  # return training and testing data
  return(trainX.values, trainy.values, testX.values, testy.values)

In [0]:
#Can put data link here:
url="https://drive.google.com/uc?id=1e-SHtWDtg4mD_th3_4Jmq9r1iiQC32wT"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')), sep = ' ')
my_list = c['id'].tolist()
unique_rnas = []
for values in my_list:
  if values not in unique_rnas:
    unique_rnas.append(values)

#stack (1) and non-stack (0)
c = c.drop(columns = ['Unnamed: 0','base_pairing','orientation','sugar_puckering','pseudoknot'])

c = c.replace('stack',1).replace('non-stack',0)

In [0]:
#Due to keras not having f1 score, recall, and precision built in, it must be defined. Use keras backend for shortening code
from keras import backend as K

#Define recall by the ratio of true positive to possible positives. k.epsilon is a fuzzy constant used to prevent dividing by 0
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

#Define precision as the ratio of true positives to predicted positives
def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

#Define f1 score as the ratio of the product to the sum of precision and recall, scaled by 2
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
#Defining a function (ODC = One Dimensional Convolution)
def ODC(rnaid, n, cs_all):
  id = rnaid
  trainX, trainy, testX, testy = create_training_testing(cs_all, leave_out = id, neighbors = n)
  scaler = StandardScaler()
  scaler.fit(trainX)
  trainX_scaled = scaler.transform(trainX)
  testX_scaled = scaler.transform(testX)

  #The data needs to have its dimensions expanded in order to be put into the 1d convolution
  trainX = np.expand_dims(trainX, axis=2)
  testX = np.expand_dims(testX, axis=2)

  #Creating each layer. Only using one conv1d layer because the vector is small and the patterns may not be relevant due to arbitrary ordering of chemical shifts
  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=(trainX.shape[1],trainX.shape[2])))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(100, activation='relu'))
  model.add(Dense(50, activation='relu'))
  model.add(Dense(trainX.shape[2], activation='softmax'))

  #Compile the model, and define an appropriate loss function for binary classification, and define metrics using previous functions
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m,precision_m, recall_m])

  #Number of epochs set to 5 due to quick convergence in graph convolution of a relatively short input vector
  model.fit(trainX, trainy, epochs=3)
  accuracy = model.evaluate(testX, testy)

  #Just return f1 score
  return accuracy[2]

In [25]:
#Run for neighbor number = 1, creating full dataset of RNAS and neighbor chemical shifts

n = 1
cs_all = get_cs_features_rna_all(c, n)
data_list = []
for rna in unique_rnas:
  data_list.append(ODC(rna, n, cs_all))

#Take the average of all the f1 scores
mean_score = sum(data_list)/len(data_list)
mean_score

Epoch 1/3
3024/3024 [==============================] - 4s 1ms/step - loss: 1.1885 - acc: 0.8056 - f1_m: 0.8711 - precision_m: 0.8393 - recall_m: 0.9133
Epoch 2/3
3024/3024 [==============================] - 1s 261us/step - loss: 0.4348 - acc: 0.8578 - f1_m: 0.9212 - precision_m: 0.8694 - recall_m: 0.9831
Epoch 3/3
44/44 [==============================] - 1s 30ms/step
Epoch 1/3
3038/3038 [==============================] - 4s 1ms/step - loss: 2.1073 - acc: 0.8644 - f1_m: 0.9253 - precision_m: 0.8676 - recall_m: 0.9945
Epoch 2/3
3038/3038 [==============================] - 1s 263us/step - loss: 2.1201 - acc: 0.8670 - f1_m: 0.9278 - precision_m: 0.8670 - recall_m: 1.0000
Epoch 3/3
30/30 [==============================] - 1s 45ms/step
Epoch 1/3
3037/3037 [==============================] - 4s 1ms/step - loss: 2.0998 - acc: 0.8683 - f1_m: 0.9286 - precision_m: 0.8683 - recall_m: 1.0000
Epoch 2/3
3037/3037 [==============================] - 1s 243us/step - loss: 2.0998 - acc: 0.8683 - f1_m: 0.

0.9201851690661483